# Cleaning Scraped Indeed Data

## Load in Scraped Data

In [26]:
import pandas as pd

# file_name = '2019-09-02_indeed-ds-postings.csv'
file_name = '2019-09-03_indeed-ds-postings.csv'
# # df = pd.read_csv('2019-09-02indeed_ds_postings.csv')
# names = ['UID', 'job_title', 'company_name', 'city', 'state', \
#          'zipcode', 'neighborhood', 'description', 'salary', 'link']
# df = pd.read_csv(file_name, index_col='UID', skiprows=1, names=names)
# names = ['job_title', 'company_name', 'city', 'state', \
#          'zipcode', 'neighborhood', 'description', 'salary_range', 'link']
names = ['job_title', 'company_name', 'location', 'neighborhood', 'description', 'salary_range', 'link']
df = pd.read_csv(file_name, index_col=0, skiprows=1, names=names)

## 1. Getting Rid of Duplicates

In [27]:
print('Before: ', df.shape)
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)
print('After: ', df.shape)

Before:  (1007, 7)
After:  (461, 7)


## 2. Cleaning Up Salary

### Looking at Salary Values

In [28]:
df['salary_range'].value_counts()

                                               431
$69,929 - $102,939 a year                        2
$20.62 an hour                                   2
$150,000 - $220,000 a year                       1
$35 - $40 an hour                                1
$117,400 - $152,000 a year                       1
$48,000 - $52,000 a year                         1
$120,000 - $150,000 a year                       1
$60,000 - $120,000 a year                        1
$50,000 - $60,000 a year                         1
Similar jobs pay $103,000 - $152,000 a year      1
$3,708 a month                                   1
Similar jobs pay $76,000 - $112,000 a year       1
$55,936 - $81,432 a year                         1
$90 - $100 a day                                 1
$50,794 - $71,864 a year                         1
$15 - $20 an hour                                1
$63,189 - $92,072 a year                         1
Similar jobs pay $49,000 - $64,000 a year        1
$100,000 a year                

### Getting Rid of Excess Symbols

In [29]:
df['salary_range'] = df['salary_range'].str.replace(',', '')
df['salary_range'] = df['salary_range'].str.replace('$', '')
df['salary_range'] = df['salary_range'].str.replace('Similar jobs pay ', '')

###  Saving Pay Period

In [30]:
df.loc[df['salary_range'].str.endswith('a year'), 'period'] = 'yearly'
df.loc[df['salary_range'].str.endswith('a month'), 'period'] = 'monthly'
df.loc[df['salary_range'].str.endswith('a week'), 'period'] = 'weekly'
df.loc[df['salary_range'].str.endswith('a day'), 'period'] = 'daily'
df.loc[df['salary_range'].str.endswith('an hour'), 'period'] = 'hourly'

### Getting Rid of String in Salary

In [31]:
# df['salary'].apply(lambda s: s.rstrip('a year'))
df['salary_range'] = df['salary_range'].str.rstrip('a year')
df['salary_range'] = df['salary_range'].str.rstrip('a month')
df['salary_range'] = df['salary_range'].str.rstrip('a week')
df['salary_range'] = df['salary_range'].str.rstrip('a day')
df['salary_range'] = df['salary_range'].str.rstrip('an hour')

In [32]:
df.head()

,job_title,company_name,location,neighborhood,description,salary_range,link,period
0,"Statistical Genetics, Data Scientist",Camp4 Therapeutics Corporation,"Cambridge, MA",,Job Description CAMP4 is seeking a Data Scient...,,03bf439bfa53ee13,NaN
1,Data Scientist,"MIB Group, Inc.","Braintree, MA 02184",,POSITION SUMMARY: MIB is committed to providin...,,c920345674fcc072,NaN
2,Translational Medicine and Data Science Expert,Novartis,"Cambridge, MA",,20 petabytes of data. 2 million patient-years ...,,4fdbb9b5cb09e0d2,NaN
3,Computational Biologist/Data Scientist,Goldfinch Bio,"Cambridge, MA",,Goldfinch Bio is a biotechnology company that ...,,f14bc6dec8b4f60f,NaN
4,Principal Data Scientist,Verizon,"Boston, MA 02109",Central area,What you’ll be doing... As a Principal Data Sc...,,8d13390a342786e0,NaN


### Converting Salary Range String to Average Annual Float

In [33]:
df['salary_range'].value_counts()
df['period'].value_counts()

yearly     24
hourly      4
monthly     1
daily       1
Name: period, dtype: int64

In [34]:
periods_in_workyear = {'yearly':1, 'monthly':12, 'weekly':52, 'daily':261, 'hourly': 2088}
import numpy as np

def get_salary_avg(row):
    sal_range = row['salary_range']
    sal = sal_range.split('-')
    period = row['period']
    try:
        return np.mean( [int(x) for x in sal] ) * periods_in_workyear[period]
    except:
        return np.nan
#     try:
#         avg_salary = (float(sal[0]) + float(sal[1])) / 2
#         avg_salary_yearly = avg_salary * periods_in_workyear[period]
#         return int(avg_salary_yearly)
#     except:
#         try:
#             return int(float(sal_range) * periods_in_workyear[period])
#         except:
#             try:
#                 return int(sal_range)
#             except:
#                 return np.nan

df['annual_salary'] = df.apply(get_salary_avg, axis=1)

In [35]:
# df['annual_salary'].value_counts()
df['annual_salary'].sort_values()

8       24795.0
92      36540.0
301     44496.0
210     50000.0
52      55000.0
391     56500.0
167     57730.5
203     59750.0
146     61329.0
187     68684.0
38      72787.0
430     77630.5
207     78300.0
372     86434.0
397     86434.0
440     90000.0
65      92000.0
99      94000.0
6       97500.0
122    100000.0
373    118572.0
25     125000.0
274    126000.0
91     127500.0
44     130000.0
130    134700.0
109    135000.0
28     185000.0
0           NaN
1           NaN
         ...   
429         NaN
431         NaN
432         NaN
433         NaN
434         NaN
435         NaN
436         NaN
437         NaN
438         NaN
439         NaN
441         NaN
442         NaN
443         NaN
444         NaN
445         NaN
446         NaN
447         NaN
448         NaN
449         NaN
450         NaN
451         NaN
452         NaN
453         NaN
454         NaN
455         NaN
456         NaN
457         NaN
458         NaN
459         NaN
460         NaN
Name: annual_salary, Len

In [36]:
df.loc[ df['annual_salary']==24795 ]

,job_title,company_name,location,neighborhood,description,salary_range,link,period,annual_salary
8,Sr. Data Scientist,Cubic IT,"Boston, MA",,Job SummaryLooking for an experienced Machine ...,90 - 100,8b0554b4567cb6b0,daily,24795.0


In [37]:
df.head(10)
# df

,job_title,company_name,location,neighborhood,description,salary_range,link,period,annual_salary
0,"Statistical Genetics, Data Scientist",Camp4 Therapeutics Corporation,"Cambridge, MA",,Job Description CAMP4 is seeking a Data Scient...,,03bf439bfa53ee13,NaN,NaN
1,Data Scientist,"MIB Group, Inc.","Braintree, MA 02184",,POSITION SUMMARY: MIB is committed to providin...,,c920345674fcc072,NaN,NaN
2,Translational Medicine and Data Science Expert,Novartis,"Cambridge, MA",,20 petabytes of data. 2 million patient-years ...,,4fdbb9b5cb09e0d2,NaN,NaN
3,Computational Biologist/Data Scientist,Goldfinch Bio,"Cambridge, MA",,Goldfinch Bio is a biotechnology company that ...,,f14bc6dec8b4f60f,NaN,NaN
4,Principal Data Scientist,Verizon,"Boston, MA 02109",Central area,What you’ll be doing... As a Principal Data Sc...,,8d13390a342786e0,NaN,NaN
5,Data Scientist / Machine Learning Architect / ...,Profitect Inc.,"Burlington, MA",,Profitect’s Research and Development team is l...,,b7ae218bbb0b2a50,NaN,NaN
6,Data Scientist (Full-Time),proton.ai,"Boston, MA",,*Job Description Data Scientist (Full-Time)Tea...,75000 - 120000,8cacfaa3c21d0129,yearly,97500.0
7,"Data Scientist (Intern, Part-Time)",proton.ai,"Boston, MA",,"Data Scientist (Intern, Part-Time)Team: Data ...",,1a087273ecb3d9e3,NaN,NaN
8,Sr. Data Scientist,Cubic IT,"Boston, MA",,Job SummaryLooking for an experienced Machine ...,90 - 100,8b0554b4567cb6b0,daily,24795.0
9,Data Scientist,Park Jockey,"Boston, MA",,Who You’ll Work For REEF Technology is the eco...,,0cc7c0afb827e835,NaN,NaN


### Switch the Columns in Case We Use it Later

In [38]:
cols = df.columns.tolist()
cols

['job_title',
 'company_name',
 'location',
 'neighborhood',
 'description',
 'salary_range',
 'link',
 'period',
 'annual_salary']

In [39]:
cols[5], cols[-1] = cols[-1], cols[5]
cols

['job_title',
 'company_name',
 'location',
 'neighborhood',
 'description',
 'annual_salary',
 'link',
 'period',
 'salary_range']

In [40]:
df = df[cols]
df.head(10)

,job_title,company_name,location,neighborhood,description,annual_salary,link,period,salary_range
0,"Statistical Genetics, Data Scientist",Camp4 Therapeutics Corporation,"Cambridge, MA",,Job Description CAMP4 is seeking a Data Scient...,NaN,03bf439bfa53ee13,NaN,
1,Data Scientist,"MIB Group, Inc.","Braintree, MA 02184",,POSITION SUMMARY: MIB is committed to providin...,NaN,c920345674fcc072,NaN,
2,Translational Medicine and Data Science Expert,Novartis,"Cambridge, MA",,20 petabytes of data. 2 million patient-years ...,NaN,4fdbb9b5cb09e0d2,NaN,
3,Computational Biologist/Data Scientist,Goldfinch Bio,"Cambridge, MA",,Goldfinch Bio is a biotechnology company that ...,NaN,f14bc6dec8b4f60f,NaN,
4,Principal Data Scientist,Verizon,"Boston, MA 02109",Central area,What you’ll be doing... As a Principal Data Sc...,NaN,8d13390a342786e0,NaN,
5,Data Scientist / Machine Learning Architect / ...,Profitect Inc.,"Burlington, MA",,Profitect’s Research and Development team is l...,NaN,b7ae218bbb0b2a50,NaN,
6,Data Scientist (Full-Time),proton.ai,"Boston, MA",,*Job Description Data Scientist (Full-Time)Tea...,97500.0,8cacfaa3c21d0129,yearly,75000 - 120000
7,"Data Scientist (Intern, Part-Time)",proton.ai,"Boston, MA",,"Data Scientist (Intern, Part-Time)Team: Data ...",NaN,1a087273ecb3d9e3,NaN,
8,Sr. Data Scientist,Cubic IT,"Boston, MA",,Job SummaryLooking for an experienced Machine ...,24795.0,8b0554b4567cb6b0,daily,90 - 100
9,Data Scientist,Park Jockey,"Boston, MA",,Who You’ll Work For REEF Technology is the eco...,NaN,0cc7c0afb827e835,NaN,


## 3. Splitting Location into City, State, and Zipcode
Define functions to apply each row

In [41]:
def get_zipcode(location):
    zipcode = ' '
    temp = [ s for s in location.split() if s.isdigit() ]
    if temp:
        zipcode = temp.pop()
    return zipcode

def get_city_and_state(location):
    city_state = location.split(', ')
    state = city_state.pop()
    city = city_state.pop()
    return city, state

def parse_location_info(row):
    location = row['location']
    
    zipcode = get_zipcode(location)
    
    location = location.strip(zipcode)
    city, state = get_city_and_state(location)
    
    row['city'] = city
    row['state'] = state
    row['zipcode'] = zipcode
    return row

Apply the parsing functions

In [42]:
df = df.apply(parse_location_info, axis=1)
df.head()

,job_title,company_name,location,neighborhood,description,annual_salary,link,period,salary_range,city,state,zipcode
0,"Statistical Genetics, Data Scientist",Camp4 Therapeutics Corporation,"Cambridge, MA",,Job Description CAMP4 is seeking a Data Scient...,NaN,03bf439bfa53ee13,NaN,,Cambridge,MA,
1,Data Scientist,"MIB Group, Inc.","Braintree, MA 02184",,POSITION SUMMARY: MIB is committed to providin...,NaN,c920345674fcc072,NaN,,Braintree,MA,02184
2,Translational Medicine and Data Science Expert,Novartis,"Cambridge, MA",,20 petabytes of data. 2 million patient-years ...,NaN,4fdbb9b5cb09e0d2,NaN,,Cambridge,MA,
3,Computational Biologist/Data Scientist,Goldfinch Bio,"Cambridge, MA",,Goldfinch Bio is a biotechnology company that ...,NaN,f14bc6dec8b4f60f,NaN,,Cambridge,MA,
4,Principal Data Scientist,Verizon,"Boston, MA 02109",Central area,What you’ll be doing... As a Principal Data Sc...,NaN,8d13390a342786e0,NaN,,Boston,MA,02109


## 4. Set Description to Lower Case
We're going to do some analysis on the description column. Let's make all of the words lower case so there's no difference between a word that starts a sentence to one that appears anywhere else.

In [44]:
df['description'] = df['description'].str.lower()
df.head()

,job_title,company_name,location,neighborhood,description,annual_salary,link,period,salary_range,city,state,zipcode
0,"Statistical Genetics, Data Scientist",Camp4 Therapeutics Corporation,"Cambridge, MA",,job description camp4 is seeking a data scient...,NaN,03bf439bfa53ee13,NaN,,Cambridge,MA,
1,Data Scientist,"MIB Group, Inc.","Braintree, MA 02184",,position summary: mib is committed to providin...,NaN,c920345674fcc072,NaN,,Braintree,MA,02184
2,Translational Medicine and Data Science Expert,Novartis,"Cambridge, MA",,20 petabytes of data. 2 million patient-years ...,NaN,4fdbb9b5cb09e0d2,NaN,,Cambridge,MA,
3,Computational Biologist/Data Scientist,Goldfinch Bio,"Cambridge, MA",,goldfinch bio is a biotechnology company that ...,NaN,f14bc6dec8b4f60f,NaN,,Cambridge,MA,
4,Principal Data Scientist,Verizon,"Boston, MA 02109",Central area,what you’ll be doing... as a principal data sc...,NaN,8d13390a342786e0,NaN,,Boston,MA,02109


## Get Rid of the Columns We Don't Need and Rearrange

In [45]:
# df = df.drop(['salary_range', 'period'], axis=1)
# df = df
# df.head()
cols = df.columns.tolist()
cols

['job_title',
 'company_name',
 'location',
 'neighborhood',
 'description',
 'annual_salary',
 'link',
 'period',
 'salary_range',
 'city',
 'state',
 'zipcode']

In [46]:
cols = ['job_title',
        'company_name',
        'annual_salary',
        'city',
        'state',
        'zipcode',
        'neighborhood',
        'description',
        'link',
]
df = df[cols]

In [47]:
df.head(10)

,job_title,company_name,annual_salary,city,state,zipcode,neighborhood,description,link
0,"Statistical Genetics, Data Scientist",Camp4 Therapeutics Corporation,NaN,Cambridge,MA,,,job description camp4 is seeking a data scient...,03bf439bfa53ee13
1,Data Scientist,"MIB Group, Inc.",NaN,Braintree,MA,02184,,position summary: mib is committed to providin...,c920345674fcc072
2,Translational Medicine and Data Science Expert,Novartis,NaN,Cambridge,MA,,,20 petabytes of data. 2 million patient-years ...,4fdbb9b5cb09e0d2
3,Computational Biologist/Data Scientist,Goldfinch Bio,NaN,Cambridge,MA,,,goldfinch bio is a biotechnology company that ...,f14bc6dec8b4f60f
4,Principal Data Scientist,Verizon,NaN,Boston,MA,02109,Central area,what you’ll be doing... as a principal data sc...,8d13390a342786e0
5,Data Scientist / Machine Learning Architect / ...,Profitect Inc.,NaN,Burlington,MA,,,profitect’s research and development team is l...,b7ae218bbb0b2a50
6,Data Scientist (Full-Time),proton.ai,97500.0,Boston,MA,,,*job description data scientist (full-time)tea...,8cacfaa3c21d0129
7,"Data Scientist (Intern, Part-Time)",proton.ai,NaN,Boston,MA,,,"data scientist (intern, part-time)team: data ...",1a087273ecb3d9e3
8,Sr. Data Scientist,Cubic IT,24795.0,Boston,MA,,,job summarylooking for an experienced machine ...,8b0554b4567cb6b0
9,Data Scientist,Park Jockey,NaN,Boston,MA,,,who you’ll work for reef technology is the eco...,0cc7c0afb827e835


## Save Cleaned Data

In [48]:
name, ext = file_name.split('.')
df.to_csv(name+'_cleaned.' + ext, encoding='utf-8')

In [49]:
# df['salary_period'] = df['salary'].apply( lambda s : s.endswith('a year') )
# df[df['salary_period'] == True] = 'year'
# df[ lambda s: df.salarys.endswith('a year') ] 
# df['hi'] = df.salary.apply(lambda s: s.endswith('a year'))


# df
# df.loc[0,'description']

In [50]:
df.loc[ df['company_name'] == 'Cambridge Innovation Center' ]

,job_title,company_name,annual_salary,city,state,zipcode,neighborhood,description,link
97,Part-time Research Analyst - CIC Captains of I...,Cambridge Innovation Center,NaN,Cambridge,MA,02142,East Cambridge area,"captains of innovation, located within one of ...",5b4552204b839e19


In [43]:
df.iloc[1].description

"POSITION SUMMARY: MIB is committed to providing valued-added services to customers and improving efficiency through enhanced capabilities in data and advanced analytics. As an integral part of this long-term strategy, this role is critical to improving operations, expanding product offerings, and servicing the life insurance industry.MINIMUM QUALIFICATIONS AND REQUIREMENTS:Education: A master’s degree in a quantitative field such as statistics, mathematics, computer science, engineering, and physics. Ph.D is a plus.Experience: 2 – 6 years of relevant experience in insurance, banking or other financial industriesSkills:o Must be proficient in SQLo Must be expert in at least one of the following - R, Python, SASo Preferred additional skills include VBA, EXCEL, ACCESS, SQL Server, DB2, PostgreSQL, UNIX, LINUX, C, C++, JAVA, JavaScript, SparkIn-depth knowledge of statistical techniques including:o GLM (multiple regression, logistic regression, log-linear regression, and variable selection

In [51]:
df.iloc[1].description

"position summary: mib is committed to providing valued-added services to customers and improving efficiency through enhanced capabilities in data and advanced analytics. as an integral part of this long-term strategy, this role is critical to improving operations, expanding product offerings, and servicing the life insurance industry.minimum qualifications and requirements:education: a master’s degree in a quantitative field such as statistics, mathematics, computer science, engineering, and physics. ph.d is a plus.experience: 2 – 6 years of relevant experience in insurance, banking or other financial industriesskills:o must be proficient in sqlo must be expert in at least one of the following - r, python, saso preferred additional skills include vba, excel, access, sql server, db2, postgresql, unix, linux, c, c++, java, javascript, sparkin-depth knowledge of statistical techniques including:o glm (multiple regression, logistic regression, log-linear regression, and variable selection

In [25]:
df.iloc[1].link

'c920345674fcc072'